Now,we have both uncleaned and cleaned dataset's .let's see how a fine tuned hate speech model performs in both the scenarios.

### Importing the libraries

In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification,pipeline#for model in huggingface
from sklearn.metrics import accuracy_score, f1_score #my evaluation metrics

### Let's start with the uncleaned data

In [2]:
# Loading the dataset
df = pd.read_excel("final_tweets.xlsx")

In [3]:
# 'Text' contains the tweets and 'Annotator-1' contains the main labels(my labels),i wil convert them into list for easy processing
texts = df['Text'].tolist()
true_labels = df['Annotator-1'].tolist()

### Loading the model
https://huggingface.co/Hate-speech-CNERG/dehatebert-mono-english
and refered from the model documentation

In [4]:
# Load HateBERT model and tokenizer
model_name = "Hate-speech-CNERG/dehatebert-mono-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)#loading the already coded tokenizer from hugging face
model = AutoModelForSequenceClassification.from_pretrained(model_name)#loading the model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
# Define the classification pipeline(AS in model documentation)
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, return_all_scores=True)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


### Model training

Here, I follow the **zero-shot** approach. I didn't fine tuned the model and will directly feed my uncleaned dataset to the model for the training and let's see how it performs.



I have given the label names here as HATE,NON HATE as the model is pre-trained and fine-tuned based on those labels;such that model understands my case. From the official documentation of the model, i can clearly state that my labels matches the model's labels as they are written on the similar pattern of annotation guidelines.

In [7]:
# Performming th classification
#intialise the empty list to store the predictions
predictions = []
#starting a for loop in order to iterate each sample
for text in df['Text']:
  #using the classifier (model pipeline) to predict scores for the text
    result = classifier(text)

    # Access the first element of the outer list given by the pipeline.Since the outer list is quite unnecessary when processing a single input, result[0] extracts the inner list containing the predictions.
    result = result[0]  # This gives you the inner list;without this ,it gave me errors for the label extraction process.

    try:
        # Extract the score for 'NON_HATE' and 'HATE'(Model's pre defined labels), it results a dictionaries where ,each has a 'label' and 'score'
        non_hate_score = next(item['score'] for item in result if item['label'] == "NON_HATE")#NEUTRAL label
        hate_score = next(item['score'] for item in result if item['label'] == "HATE")#HATE/OFFENSIVE label
        #if a key error occurs, i have got this argument to ignore that , this generally happened to a glitch. i searched for this in stackoverflow,it suggested me this.
    except KeyError as e:
        print(f"Unexpected output structure: {result}")
        raise e
#now, it gave scores like for example hate_score:0.78,non_hate 0.22, so i have assigned a condition in which if hate score>non hate, it will label as '1',otherwise '0'which matches my labelling scenario.
    # Assign the predicted label (1 for HATE, 0 for NON_HATE)
    pred_label = 1 if hate_score > non_hate_score else 0
    predictions.append(pred_label)

# Add predictions to the dataframe
df['predicted_label'] = predictions



In [8]:
#Calculate evaluation metrics
# calculate accuracy
accuracy = accuracy_score(true_labels, df['predicted_label'])

# calculate F1 score (binary classification, average='binary' by default)
f1 = f1_score(true_labels, df['predicted_label'])

# Print the results
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

Accuracy: 0.7066666666666667
F1 Score: 0.65625


In [9]:
df.head(15)

,Text,Annotator-1,Annotator-2,predicted_label
0,hey new followers *waving* welcome ion be talk...,1,1,0
1,The Taliban want a lot for an extension. The ...,0,0,0
2,@USER 😂😂😂😂😂you retarded,1,1,1
3,@USER A little nigga once told me why you cal...,1,1,1
4,@USER don't name everything by using racist! ...,0,0,0
5,oh my gosh. that's got to be difficult.Take c...,0,0,0
6,Hell yeah real madrid plays today; I'm soo pum...,0,0,0
7,@USER my review got downvotes because GamerGa...,1,0,0
8,Coronavirus and the global Panic..demic #Pande...,0,0,1
9,@USER ur right I'm retarded,1,1,1


In [10]:
#saving the predicted labels to the dataset
df.to_excel("final_uncleaned_predicted__data.xlsx", index=False)

### Let's look at the  cleaned data and perform zero-shot on that
(code is same for the below part,i just changed the dataset)
---



In [11]:
# Loading the dataset
data = pd.read_excel("final_tweets_cleaned.xlsx")
ctexts = data['Text'].tolist()
ctrue_labels = data['Annotator-1'].tolist()

### Model Training

In [12]:

#intialise the empty list to store the predictions
predictions = []
#starting a for loop in order to iterate each sample
for text in data['Text']:
  #using the classifier (model pipeline) to predict scores for the text
    result = classifier(text)
# Access the first element of the outer list given by the pipeline.Since the outer list is quite unnecessary when processing a single input, result[0] extracts the inner list containing the predictions.
    result = result[0]  # This gives you the inner list;without this ,it gave me errors for the label extraction process.

    try:
        # Extract the score for 'NON_HATE' and 'HATE'(Model's pre defined labels), it results a dictionaries where ,each has a 'label' and 'score'
        non_hate_score = next(item['score'] for item in result if item['label'] == "NON_HATE")#NEUTRAL label
        hate_score = next(item['score'] for item in result if item['label'] == "HATE")#HATE/OFFENSIVE label
  #if a key error occurs, i have got this argument to ignore that , this generally happened to a glitch. i searched for this in stackoverflow,it suggested me this.
    except KeyError as e:
        print(f"Unexpected output structure: {result}")
        raise e

    #now, it gave scores like for example hate_score:0.78,non_hate 0.22, so i have assigned a condition in which if hate score>non hate, it will label as '1',otherwise '0'which matches my labelling scenario.
    # Assign the predicted label (1 for HATE, 0 for NON_HATE)
    pred_label = 1 if hate_score > non_hate_score else 0
    predictions.append(pred_label)

# Add predictions to the dataframe
data['predicted_label'] = predictions


In [13]:
accuracy = accuracy_score(ctrue_labels, data['predicted_label'])

# Calculate F1 score (binary classification, average='binary' by default)
f1 = f1_score(ctrue_labels, data['predicted_label'])

# Print the results
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

Accuracy: 0.7933333333333333
F1 Score: 0.7350427350427351


In [15]:
data.head(15)

,Text,Annotator-1,Annotator-2,predicted_label
0,hey new followers waving welcome ion be talkin...,1,1,1
1,the taliban want a lot for an extension the bi...,0,0,0
2,you retarded,1,1,1
3,a little nigga once told me why you called me ...,1,1,0
4,dont name everything by using racist just tell...,0,0,0
5,oh my gosh thats got to be difficulttake care ...,0,0,0
6,hell yeah real madrid plays today im soo pumpe...,0,0,0
7,my review got downvotes because gamergate dogp...,1,0,0
8,coronavirus and the global panicdemic 19,0,0,0
9,ur right im retarded,1,1,1


store the predicted labels to the new dataset

In [16]:
df.to_excel("final_cleaned_predicted__data.xlsx", index=False)

From the above training with the zero-shot methodology on un-cleaned and cleaned data and by abserving the evaluation metrics , i observed that the model performed very well with the  cleaned data. The accuracy and F1 scores are quite-mid as the model is not finetuned based on my data. so , it generally happens.

Thus,i conclude my project.

## THANK YOU